In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pivottablejs import pivot_ui
from sklearn.compose import ColumnTransformer

In [10]:
pip install --upgrade google-api-python-client

SyntaxError: invalid syntax (1030666562.py, line 1)

In [6]:
pip install --upgrade google-auth-oauthlib google-auth-httplib2

   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# -*- coding: utf-8 -*-

import os
import googleapiclient.discovery
import csv

DEVELOPER_KEY = "INSERT_YOUR_DEV_KEY"
VIDEO_ID = "vovkzbtYBC8"

# Функция для скачивания корневых комментариев
def youtube(nextPageToken=None):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"    

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="id,snippet",
        maxResults=100,
        pageToken=nextPageToken,
        videoId=VIDEO_ID
    )
    response = request.execute()
    return response

# Функция для скачивания реплаев на комментарии
def youtubechild(NextParentId, nextPageToken=None):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.comments().list(
        part="id,snippet",
        maxResults=100,
	pageToken=nextPageToken,
        parentId=NextParentId
    )
    response = request.execute()
    return response

# Главная функция
def main():
    # Скачиваем комментарии
    print('download comments')
    response = youtube()
    items = response.get("items")
    nextPageToken = response.get("nextPageToken") # скачивается порциями, на каждую следующую выдаётся указатель
    i=1
    while nextPageToken is not None:
        print(str(i*100)) # показываем какая сотня комментариев сейчас скачивается
        response = youtube(nextPageToken)
        nextPageToken = response.get("nextPageToken")
        items = items + response.get("items")
        i+=1

    print(len(items)) # Отображаем количество скачаных комментариев

    # Скачиваем реплаи на комментарии
    print('download replies')
    replies = []
    for line in items: # Проходим по корневым комментам
        if line.get("snippet").get("totalReplyCount") > 0: # если есть реплаи
            print(line.get("snippet").get("totalReplyCount")) # показываем сколько реплаев будет подгружено
            response = youtubechild(line.get("snippet").get("topLevelComment").get("id"))
            replies = replies + response.get("items")
            nextPageToken = response.get("nextPageToken")
            i=1
            while nextPageToken is not None: # догружаем реплаи, если есть ещё порции
                response = youtubechild(line.get("snippet").get("topLevelComment").get("id"), nextPageToken)
                nextPageToken = response.get("nextPageToken")
                replies = replies + response.get("items")
                i+=1

    print(len(replies)) # Отображаем количество скачаных реплаев

    # Сохраняем комментарии и реплаи на них в файл csv
    print("Open csv file")
    with open('youtuberesults.csv', 'w', encoding="utf-8") as csv_file:  #конструкция with, чтобы файл закрылся автоматом после всех команд
        writer = csv.writer(csv_file, quoting=csv.QUOTE_ALL, lineterminator='\r') # использовал двойные кавычки и разделитель запятую, такой формат отлично открывается через LibreOffice Calc

        # Заголовки столбцов
        row = [
              'etag'
            , 'parentid'
            , 'id'
            , 'textDisplay'
            , 'textOriginal'
            , 'authorDisplayName'
            , 'authorProfileImageUrl'
            , 'authorChannelUrl'
            , 'authorChannelId'
            , 'likeCount'
            , 'publishedAt'
            , 'updatedAt'
        ]
        print("Start write in csv")  
        writer.writerow(row) # Записываем заголовки в файл
    
        # Сохраняем комментарии
        print("Write comments in csv")  
        for line in items:
            topLevelComment = line.get("snippet").get("topLevelComment")
            # бывает, что у пользователя нет канала, поэтому для него отдельная конструкция
            if topLevelComment.get('snippet').get('authorChannelId') is not None:
                authorChannelId = topLevelComment.get('snippet').get('authorChannelId').get('value')
            else:
                authorChannelId = ''
            row = [
                  topLevelComment.get('etag')
                , topLevelComment.get('id')
                , topLevelComment.get('id')
                , topLevelComment.get('snippet').get('textDisplay')
                , topLevelComment.get('snippet').get('textOriginal')
                , topLevelComment.get('snippet').get('authorDisplayName')
                , topLevelComment.get('snippet').get('authorProfileImageUrl')
                , topLevelComment.get('snippet').get('authorChannelUrl')
                , authorChannelId
                , topLevelComment.get('snippet').get('likeCount')
                , topLevelComment.get('snippet').get('publishedAt')
                , topLevelComment.get('snippet').get('updatedAt')
            ]
            writer.writerow(row)

        # Save replies
        print("Write replies in csv")  
        for line in replies:
            # бывает, что у пользователя нет канала, поэтому для него отдельная конструкция
            if line.get('snippet').get('authorChannelId') is not None:
                authorChannelId = line.get('snippet').get('authorChannelId').get('value')
            else:
                authorChannelId = ''
            row = [
                  line.get('etag')
                , line.get('snippet').get('parentId')
                , line.get('id')
                , line.get('snippet').get('textDisplay')
                , line.get('snippet').get('textOriginal')
                , line.get('snippet').get('authorDisplayName')
                , line.get('snippet').get('authorProfileImageUrl')
                , line.get('snippet').get('authorChannelUrl')
                , authorChannelId
                , line.get('snippet').get('likeCount')
                , line.get('snippet').get('publishedAt')
                , line.get('snippet').get('updatedAt')
            ]
            writer.writerow(row)
        
    print("done")  

if __name__ == "__main__":
    main()

download comments
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
1820

In [13]:
df = pd.read_csv("youtuberesults.csv")

In [14]:
df.head()

,etag,parentid,id,textDisplay,textOriginal,authorDisplayName,authorProfileImageUrl,authorChannelUrl,authorChannelId,likeCount,publishedAt,updatedAt
0,8afBP0g4q1VdlfL9v9qhQh5XKS0,Ugyrih7ysCq7xMawUeN4AaABAg,Ugyrih7ysCq7xMawUeN4AaABAg,Was wondering if I’m still watching an assassi...,Was wondering if I’m still watching an assassi...,@yeliseyshitik2767,https://yt3.ggpht.com/ytc/AIdro_kHr5RBTLXjqpn7...,http://www.youtube.com/@yeliseyshitik2767,UC1dFzRmJaVwejnnGVFeoQlA,0,2024-06-23T21:29:51Z,2024-06-23T21:29:51Z
1,-OveCdhEp9zYXr0zZD2wLVMdq6I,UgzJ_zwWiX3OfhTyIDh4AaABAg,UgzJ_zwWiX3OfhTyIDh4AaABAg,I really hope they redo the music to something...,I really hope they redo the music to something...,@thelayedbackguitarist9865,https://yt3.ggpht.com/wuqdVKt_NQAl1KX4139171Vv...,http://www.youtube.com/@thelayedbackguitarist9865,UC3UjnwqU2pf8WzLZ6J2vQhw,0,2024-06-23T20:29:53Z,2024-06-23T20:29:53Z
2,Saf0SdP2eTWJEVROFiHgNvTyutk,UgzU6yRCDy3f1YD9tvB4AaABAg,UgzU6yRCDy3f1YD9tvB4AaABAg,"At the end of the day, people down playing thi...","At the end of the day, people down playing thi...",@AmalaSounds,https://yt3.ggpht.com/PLb_K0I7U3DpTmOJBb5O9FWL...,http://www.youtube.com/@AmalaSounds,UCgD04o8XHpvs7rJkQP55bRA,0,2024-06-23T19:53:39Z,2024-06-23T19:53:39Z
3,Z5lSIagdBXVSGUEsEND0NiKvJ5o,UgzlUNdHg2R7uOGtFC54AaABAg,UgzlUNdHg2R7uOGtFC54AaABAg,I&#39;m a big fan of Ghost of Tsushima and I&#...,I'm a big fan of Ghost of Tsushima and I'm sti...,@AmalaSounds,https://yt3.ggpht.com/PLb_K0I7U3DpTmOJBb5O9FWL...,http://www.youtube.com/@AmalaSounds,UCgD04o8XHpvs7rJkQP55bRA,1,2024-06-23T19:44:49Z,2024-06-23T19:44:49Z
4,PlNOJm618_LVrdYTKZ4toYop4Xk,UgwypAi0WICQlskf8LR4AaABAg,UgwypAi0WICQlskf8LR4AaABAg,I don&#39;t know why but this game reminds a l...,I don't know why but this game reminds a littl...,@Hacker65555,https://yt3.ggpht.com/JfdvOKNBah9LHLbCummaVdCc...,http://www.youtube.com/@Hacker65555,UC8EVbw6ZgrLRneVzFbx3tAQ,0,2024-06-23T17:28:47Z,2024-06-23T17:32:31Z


In [17]:
df_text = df[['textDisplay', 'textOriginal', 'authorDisplayName', 'likeCount', 'publishedAt']]